# COMPANY LOCALIZATION

#### 1. Let's import the libraries that we'll use

In [126]:
import json
from functools import reduce
import numpy as np
import operator
import os
import pandas as pd
import pymongo
from pymongo import MongoClient
import requests
import sys
sys.path.append('../')
import scr.data_functions as cf

### In order to place the new offices for the company, we seleted three different locations:
### - Palais Garnier area in Paris, France.
### - Downtown Melbourne, in Australia
### - Downtown Sao Paulo, in Brazil
###  To define the best place for the company to grow, we selected 8 different criteria based on the company's employee composition and preferences: Number of airports within a radius of 20km around, commuter stations, airport shuttles, schools and vegan restaurants within a radius of 2km around, veterinaries within a radius of 3km around, Starbucks cafe's within a radius of 500m around, and basketball courts within a radius of 10km around.
###  Later on, we'll build a score assigning weights to each quantity in order to balance the preferences and then adding all quantities for each city. We will make a decision based on that score, the city with the bigger score will be the one hosting the new offices.

## GEOQUERIES

#### 2. Now, we make connection with our Mongo DB, "GeoSpatialProyect", and define one variable for each collection we have in our database

In [3]:
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [4]:
transport = db.get_collection("transport")

In [5]:
gastronomy = db.get_collection("gastronomy")

In [6]:
schools = db.get_collection("schools")

In [7]:
veterinary = db.get_collection("veterinary")

In [8]:
basketball = db.get_collection("basketball")

#### 3. We'll perform a geo query in each collection we have, for the selected citities.

### PARIS

In [9]:
paris_pg = {'type': 'Point', 'coordinates': [48.87241163769417,2.3317031342524017]}

#### - Transport queries:

In [37]:
p_transport = [cf.geoquering(c, 2000, transport, paris_pg) for c in ['Train Station', 'Bus Stop', 'Transportation Service']]

In [47]:
p_airports = cf.geoquering('Airport', 20000, transport, paris_pg)

#### - Schools queries:

In [130]:
p_schools = [cf.geoquering(c, 2000, schools, paris_pg) for c in ['Nursery School', 'Elementary School', 'High School']]

In [131]:
p_schools[0]

[{'_id': ObjectId('6134a0143681de6a29cd9ef4'),
  'name': 'École Maternelle Blanche',
  'categories': [{'id': '4f4533814b9074f6e4fb0107',
    'name': 'Nursery School',
    'pluralName': 'Nursery Schools',
    'shortName': 'Nursery School',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
     'suffix': '.png'},
    'primary': True}],
  'latitude': 48.87691559701054,
  'longitude': 2.33236698792296,
  'location': {'type': 'Point',
   'coordinates': [48.87691559701054, 2.33236698792296]}},
 {'_id': ObjectId('6134a0143681de6a29cd9efb'),
  'name': 'Ecole Maternelle Moscou',
  'categories': [{'id': '4f4533814b9074f6e4fb0107',
    'name': 'Nursery School',
    'pluralName': 'Nursery Schools',
    'shortName': 'Nursery School',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
     'suffix': '.png'},
    'primary': True}],
  'latitude': 48.880193,
  'longitude': 2.326016,
  'location': {'type': 'Point', 'coordinates': [48.880193,

#### - Gastronomy queries:

In [49]:
p_vegan = [cf.geoquering(c, 2000, gastronomy, paris_pg) for c in ['Vegetarian / Vegan Restaurant', 'Restaurant', 'Snack Place']]

In [51]:
p_starbucks = cf.geoquering('Coffee Shop', 500, gastronomy, paris_pg)

#### - Veterinary queries:

In [52]:
p_veterinarian = cf.geoquering('Veterinarian', 3000, veterinary, paris_pg)

#### - Basket queries:

In [53]:
p_basketball_court = cf.geoquering('Basketball Court', 10000, basketball, paris_pg)

In [54]:
geoqueries_paris = np.array([len(p_airports), len(p_transport[0]), (len(p_transport[1]) + len(p_transport[2])),
                    (len(p_schools[0]) + len(p_schools[1]) + len(p_schools[2])),
                    (len(p_vegan[0])+ len(p_vegan[1]) + len(p_vegan[2])),
                    len(p_veterinarian), len(p_starbucks), len(p_basketball_court)])

### MELBOURNE

In [59]:
melbourne_cbd = {'type': 'Point', 'coordinates': [144.96262186884945, -37.81227175879301]}

#### - Transport queries:

In [60]:
m_transport = [cf.geoquering(c, 2000, transport, melbourne_cbd) for c in ['Train Station', 'Bus Line', 'Transportation Service']]

In [79]:
m_airports = cf.geoquering('Airport', 20000, transport, melbourne_cbd)

#### - Schools queries:

In [77]:
m_schools = [cf.geoquering(c, 2000, schools, melbourne_cbd) for c in ['Elementary School', 'High School', 'Daycare']]

#### - Gastronomy queries:

In [85]:
m_vegan = [cf.geoquering(c, 2000, gastronomy, melbourne_cbd) for c in ['Vegetarian / Vegan Restaurant', 'Restaurant', 'Snack Place']]

In [86]:
m_starbucks = cf.geoquering('Coffee Shop', 500, gastronomy, melbourne_cbd)

#### - Veterinary queries:

In [88]:
m_veterinarian = cf.geoquering('Veterinarian', 3000, veterinary, melbourne_cbd)

#### - Basket queries:

In [90]:
m_basketball_court = cf.geoquering('Basketball Court', 10000, basketball, melbourne_cbd)

In [91]:
geoqueries_melbourne = np.array([len(m_airports), len(m_transport[0]), (len(m_transport[1]) + len(m_transport[2])),
                    (len(m_schools[0]) + len(m_schools[1]) + len(m_schools[2])),
                    (len(m_vegan[0])+ len(m_vegan[1]) + len(m_vegan[2])),
                    len(m_veterinarian), len(m_starbucks), len(m_basketball_court)])

### SAO PAULO

In [92]:
sao_paulo_zc = {'type': 'Point', 'coordinates': [-46.639408758375076, -23.533215960575276]}

#### - Transport queries:

In [108]:
sp_transport = [cf.geoquering(c, 2000, transport, sao_paulo_zc) for c in ['Train Station', 'Bus Line', 'Transportation Service']]

In [109]:
sp_airports = cf.geoquering('Airport', 20000, transport, sao_paulo_zc)

#### - Schools queries:

In [110]:
m_schools = [cf.geoquering(c, 2000, schools, sao_paulo_zc) for c in ['Nursery School', 'Elementary School', 'High School']]

#### - Gastronomy queries:

In [111]:
sp_vegan = [cf.geoquering(c, 2000, gastronomy, sao_paulo_zc) for c in ['Vegetarian / Vegan Restaurant', 'Restaurant', 'Snack Place']]

In [112]:
sp_starbucks = cf.geoquering('Coffee Shop', 500, gastronomy, sao_paulo_zc)

#### - Veterinary queries:

In [117]:
sp_veterinarian = cf.geoquering('Veterinarian', 3000, veterinary, sao_paulo_zc)

#### - Basket queries:

In [118]:
sp_basketball_court = cf.geoquering('Basketball Court', 10000, basketball, sao_paulo_zc)

In [119]:
geoqueries_saopaulo = np.array([len(sp_airports), len(sp_transport[0]), (len(sp_transport[1]) + len(sp_transport[2])),
                    (len(m_schools[0]) + len(m_schools[1]) + len(m_schools[2])),
                    (len(sp_vegan[0])+ len(sp_vegan[1]) + len(sp_vegan[2])),
                    len(sp_veterinarian), len(sp_starbucks), len(sp_basketball_court)])

### SCORING AND FINAL DECISION

#### - With our conditions, the queries results and the wieghts we built a data frame

In [120]:
weights = np.array([0.19, 0.18, 0.17, 0.14, 0.12, 0.1, 0.05, 0.05])

In [121]:
score_p = {'Conditions': ['Airports', 'Commuter Station', 'Airport Shuttle', 'Schools', 'Vegan Restaurants', 
                          'Veterinaries', 'Starbucks Cafe', 'Basketball Court'],
           'Weights': [0.19, 0.18, 0.17, 0.14, 0.12, 0.1, 0.05, 0.05],
           'Geoqueries Paris': geoqueries_paris, 'Score Paris': weights * geoqueries_paris,
           'Geoqueries Melbourne': geoqueries_melbourne, 'Score Melbourne': weights * geoqueries_melbourne,
           'Geoqueries Sao Paulo': geoqueries_saopaulo, 'Score Sao Paulo': weights * geoqueries_saopaulo}

In [122]:
score_table = pd.DataFrame(score_p)

In [123]:
suma = score_table.sum()

In [124]:
score_table.append(suma, ignore_index=True)

,Conditions,Weights,Geoqueries Paris,Score Paris,Geoqueries Melbourne,Score Melbourne,Geoqueries Sao Paulo,Score Sao Paulo
0,Airports,0.19,2,0.38,2,0.38,1,0.19
1,Commuter Station,0.18,1,0.18,7,1.26,2,0.36
2,Airport Shuttle,0.17,3,0.51,1,0.17,1,0.17
3,Schools,0.14,8,1.12,1,0.14,1,0.14
4,Vegan Restaurants,0.12,5,0.60,1,0.12,10,1.20
5,Veterinaries,0.10,11,1.10,1,0.10,5,0.50
6,Starbucks Cafe,0.05,5,0.25,7,0.35,0,0.00
7,Basketball Court,0.05,5,0.25,20,1.00,3,0.15
8,AirportsCommuter StationAirport ShuttleSchools...,1.00,40,4.39,40,3.52,23,2.71


### FINAL DECISION: As we can see in the above data frame, Paris got the biggest overall score. So the new company will have its first offices in the center of Paris.